### Layer Generation

This file converts arcgis FeatureServer data into pmtiles and uploads them to a supabase bucket.

To add a new file: Add the arcgis URL path to the `LAYER_MAP` object.


In [2]:
LAYER_MAP = {
    # "path_lines": "https://services3.arcgis.com/Stu7jwuXrnM0myT0/arcgis/rest/services/PATH_Train/FeatureServer/0",
    # "path_stations": "https://services3.arcgis.com/Stu7jwuXrnM0myT0/arcgis/rest/services/PATH_Train/FeatureServer/0",
    # "nyc_lines": "https://services5.arcgis.com/OKgEWPlJhc3vFb8C/arcgis/rest/services/MTA_Subway_Routes_Stops/FeatureServer/1",
    # "nyc_stations": "https://services5.arcgis.com/OKgEWPlJhc3vFb8C/arcgis/rest/services/MTA_Subway_Routes_Stops/FeatureServer/0",
    # "nj_light_rail_lines": "https://services6.arcgis.com/M0t0HPE53pFK525U/arcgis/rest/services/NJTransit_Light_Rail/FeatureServer/0",
    # "nj_light_rail_stations": "https://services6.arcgis.com/M0t0HPE53pFK525U/arcgis/rest/services/NJTransit_Light_Rail_Stations/FeatureServer/0",
    # "nj_rail_lines": "https://services6.arcgis.com/M0t0HPE53pFK525U/arcgis/rest/services/NJTRANSIT_RAIL_LINES_1/FeatureServer/0",
    # "nj_rail_stations": "https://services6.arcgis.com/M0t0HPE53pFK525U/arcgis/rest/services/NJTransit_Rail_Stations/FeatureServer/0",
    "nyc_bike_lanes": "https://services9.arcgis.com/gbyGLR8owy6Q4kUA/arcgis/rest/services/NYC_Bike_Lane_Network/FeatureServer/0",
}

In [3]:
import os
from supabase import create_client
import dotenv
dotenv.load_dotenv()
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_ANON_KEY")
supabase = create_client(url, key)

def upload_to_supabase(file_path, bucket_name):
    # Read the file
    with open(file_path, "rb") as file:
        # Upload the file to the specified bucket
        response = supabase.storage.from_(bucket_name).upload(file_path, file)
        return response

# Example usage
# upload_to_supabase('./layer_outputs/nj_rail_stations.pmtiles', 'your_bucket_name')

In [5]:
import requests
import json
import subprocess
import os


def arcgis_url_to_pmtiles(name, url):
    query_url = f"{url}/query"
    
    all_features = []
    offset = 0
    max_record_count = 1000  # Common default, but we'll detect it
    
    # First, get the service metadata to find the actual max record count
    metadata_response = requests.get(f"{url}?f=json")
    metadata = metadata_response.json()
    max_record_count = metadata.get('maxRecordCount', 1000)
    
    print(f"Fetching {name} with max {max_record_count} records per request...")
    
    while True:
        params = {
            "where": "1=1",
            "outFields": "*",
            "f": "geojson",
            "returnGeometry": "true",
            "resultOffset": offset,
            "resultRecordCount": max_record_count
        }
        
        response = requests.get(query_url, params=params)
        geojson_data = response.json()
        
        features = geojson_data.get('features', [])
        
        if not features:
            break
            
        all_features.extend(features)
        print(f"  Fetched {len(all_features)} features so far...")
        
        # If we got fewer features than the max, we've reached the end
        if len(features) < max_record_count:
            break
            
        offset += max_record_count
    
    # Construct complete GeoJSON
    complete_geojson = {
        "type": "FeatureCollection",
        "features": all_features
    }
    
    print(f"Total features fetched: {len(all_features)}")
    
    # Save GeoJSON
    os.makedirs("./geo_layers", exist_ok=True)
    with open(f"./geo_layers/{name}.geojson", "w") as f:
        json.dump(complete_geojson, f)
    
    # Convert to PMTiles using tippecanoe
    subprocess.run(
        [
            "tippecanoe",
            "-o",
            f"./geo_layers/{name}.pmtiles",
            "-zg",
            "--drop-densest-as-needed",
            f"./geo_layers/{name}.geojson",
            "--force",
        ]
    )
    
    print(f"✓ Created {name}.pmtiles")


# Usage
arcgis_url_to_pmtiles(
    "nyc_bike_lanes",
    "https://services9.arcgis.com/gbyGLR8owy6Q4kUA/arcgis/rest/services/NYC_Bike_Lane_Network/FeatureServer/0"
)

Fetching nyc_bike_lanes with max 2000 records per request...
  Fetched 2000 features so far...
  Fetched 4000 features so far...
  Fetched 6000 features so far...
  Fetched 8000 features so far...
  Fetched 10000 features so far...
  Fetched 12000 features so far...
  Fetched 14000 features so far...
  Fetched 16000 features so far...
  Fetched 16672 features so far...
Total features fetched: 16672


For layer 0, using name "nyc_bike_lanes"
16672 features, 1844848 bytes of geometry and attributes, 590875 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
Choosing a maxzoom of -z11 for features typically 345 feet (105 meters) apart, and at least 57 feet (18 meters) apart


✓ Created nyc_bike_lanes.pmtiles


In [6]:
#This will fail if the file already exists. Must delete first.

BUCKET_NAME = "citi-bike-data-bucket"
for layer_name, arcgis_url in LAYER_MAP.items():
    arcgis_url_to_pmtiles(layer_name, arcgis_url)
    file_path = f"./geo_layers/{layer_name}.pmtiles"
    upload_response = upload_to_supabase(file_path, BUCKET_NAME)

Fetching nyc_bike_lanes with max 2000 records per request...
  Fetched 2000 features so far...
  Fetched 4000 features so far...
  Fetched 6000 features so far...
  Fetched 8000 features so far...
  Fetched 10000 features so far...
  Fetched 12000 features so far...
  Fetched 14000 features so far...
  Fetched 16000 features so far...
  Fetched 16672 features so far...
Total features fetched: 16672


For layer 0, using name "nyc_bike_lanes"
16672 features, 1844848 bytes of geometry and attributes, 590875 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
Choosing a maxzoom of -z11 for features typically 345 feet (105 meters) apart, and at least 57 feet (18 meters) apart
  97.8%  11/603/769  


✓ Created nyc_bike_lanes.pmtiles
